# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_DB = pd.read_csv("Food_DB.csv")

In [3]:
data_DB.head()

,Meal,Amount,Stars,Effort,Notes
0,Makaronilåda,6,5,5,NaN
1,Lasagne,5,5,5,NaN
2,Pizza,4,5,5,NaN
3,Bacon ärt pasta,4,5,4,NaN
4,Hotdog,2,5,1,NaN


# Main functions

In [71]:
# SETTINGS
days_in_plan = 7
starting_meal = 1
current_pos = [starting_meal, 1]
meals_left = 0
current_meal = None
leftover_meal = None
leftover_amount = 0
previous_meals = []
single_lunch_days = [5]
ready_lunch_days = [3,4]

In [49]:
# Create probability
data_DB["Probability"] = ((data_DB["Stars"] / 5) * 0.75) + ((1 - 0.2 * (data_DB["Effort"] - 1)) * 0.25)

In [50]:
# Create mealplan_DB
mealplan_DB = pd.DataFrame()
mealplan_DB["meal_of_day"] = ["lunch", "dinner"]
days_in_plan_range = range(0,days_in_plan)
for i in days_in_plan_range:
    mealplan_DB[i + 1] = 0

In [51]:
mealplan_DB.head()

,meal_of_day,1,2,3,4,5,6,7
0,lunch,0,0,0,0,0,0,0
1,dinner,0,0,0,0,0,0,0


In [52]:
def populate_meal(meal):
    global meals_left
    
    print(current_pos)
    print(current_meal)
    print(meals_left)
    print(previous_meals)
    
    if current_meal not in previous_meals:
        previous_meals.append(current_meal)
    
    
    if meals_left >= 2: 
        mealplan_DB.loc[current_pos[0], current_pos[1]] = meal
        meals_left = meals_left - 2
        shift_forward()
    elif meals_left == 1:
        old_meal = current_meal
        get_next_meal()
        mealplan_DB.loc[current_pos[0], current_pos[1]] = f"{old_meal} & {current_meal}"
        meals_left = meals_left - 2
        shift_forward()
    elif meals_left <= 0:
        get_next_meal()

In [68]:
def shift_forward():
    # If lunch switch to dinner
    if current_pos[0] == 0:
        current_pos[0] = 1
    # If dinner switch to lunch and move day forward
    else:
        current_pos[0] = 0
        current_pos[1] = current_pos[1] + 1

def get_next_meal():
    global current_meal
    unique_meal = False
    while unique_meal == False:
        if current_meal not in previous_meals or not previous_meals:
            unique_meal = True
        else:
            current_meal = get_random_meal()        

def get_random_meal():
    global meals_left
    new_meal = data_DB.sample()
    meal_name = new_meal["Meal"].iloc[0]
    meals_left = meals_left + new_meal["Amount"].iloc[0]
    return meal_name

def check_previous_meals():
    if current_meal not in previous_meals:
        previous_meals.append(current_meal)

In [66]:
populate_meal(current_meal)
mealplan_DB.head()

[0, 5]
Gnocchi tomat
1
[None, 'Fish & Chips', 'Knackkorvssås', 'Maletköttsås', 'Gnocchi tomat']


,meal_of_day,1,2,3,4,5,6,7
0,lunch,Fish & Chips,Maletköttsås,Maletköttsås,Gnocchi tomat,Gnocchi tomat & Salt paj,0,0
1,dinner,Knackkorvssås,Maletköttsås,Gnocchi tomat,Gnocchi tomat,0,0,0


In [9]:
def check_one_lunch():
    # Checks if current position is a single lunch meal, TRUE if yes
    if current_pos[0] == 0:
        current_day = current_pos[1]
        if current_day in single_lunch_days:
            return True
        else:
            return False
    else:
        return False
        
def check_ready_lunch_needed():
    # Checks if NEXT lunch needs to be made during dinner on previous day, TRUE if yes
    if current_pos[0] == 1:
        next_day = current_pos[1] + 1
        if next_day in ready_lunch_days:
            return True
        else:
            return False
    else:
        return False
    
def check_one_lunch_needed():
    # Checks if NEXT lunch should only be one meal during previous dinner, TRUE if yes
    if current_pos[0] == 1:
        next_day = current_pos[1] + 1
        if next_day in single_lunch_days:
            return True
        else:
            return False
    else:
        return False

In [ ]:
print(check_one_lunch())
print(check_ready_lunch_needed())
print(check_one_lunch_needed())

In [72]:
def get_next_meal(minimum_meals):
    global current_meal
    unique_meal = False
    enough_meals = False
    while unique_meal == False and enough_meals == False:
        if current_meal not in previous_meals or not previous_meals and meals_left >= minimum_meals:
            unique_meal = True
            enough_meals = True
        else:
            current_meal = get_random_meal()   

In [ ]:
print(current_meal)
print(meals_left)
print(get_next_meal(4))

In [69]:
def generate_meal_plan():
    
    check_previous_meals()
    
    if meals_left <= 0 and leftover_amount <= 0:
        if ready_lunch_needed():
            get_next_meal(4)
        else:
            get_next_meal(0)
    elif meals_left <= 0 and leftover_amount == 1:
        if ready_lunch_needed():
            get_next_meal(3)
        else:
            get_next_meal(0)
    elif meals_left <= 0 and leftover_amount >= 2:
        if ready_lunch_needed():
            get_next_meal(2)
    